# LangChain + Agent
**2024 삼성전자 무선사업부 생성형 AI 교육**

1. 기본 패키지 설정
2. Tool 설정
3. Agent + LLM 연결

In [ ]:
# 1. 패키지 설치

%pip install google-search-results
%pip install wikipedia
%pip install langchain_experimental

In [ ]:
# 2. 기본 패키지 설정 & AzureOpenAI 환경설정

import os

from config_azure import (
    AZURE_OPENAI_API_VERSION,
    AZURE_OPENAI_ENDPOINT,
    AZURE_OPENAI_KEY,
    SERPAPI_API_KEY # google search
)
# lagchain 사용시 반드시 환경변수 값 사용!!
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION

os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY

In [ ]:
# 2. Azure OpenAI LLM

from langchain_openai import AzureOpenAI

completion_llm = AzureOpenAI(
    deployment_name="gpt-35-turbo-instruct",
    max_tokens=512,
    temperature=0.1,
)


from langchain_openai import AzureChatOpenAI

chat_llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    max_tokens=512,
    temperature=0.2,
)

In [ ]:
# 3-1. Agent 생성 without LCEL

from langchain.utilities import SerpAPIWrapper
from langchain.agents import AgentExecutor
from langchain.agents import create_react_agent
from langchain.agents import load_tools
from langchain_core.prompts import PromptTemplate

tools_name = ["serpapi", "wikipedia"] # google : 최신 사건, 제품, wikipedia : 역사적인 인물, 사건, 회사, 유명한 물품
tools = load_tools(tools_name)

template = """Answer the following questions as best you can. 
You have access to the following tools: {tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(template)

search_agent = create_react_agent(chat_llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=search_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)


In [ ]:
# 3-2. Agent 실행

query1 = "갤럭시 최신 핸드폰 기종 뭐야?"
query2 = "What is the latest Galaxy phone model?"
query3 = "구글 최신 생성형 인공지능 모델을 알려줘"
query4 = "Please tell me about Google’s latest generative artificial intelligence model."

agent_executor.invoke({"input": query1})

In [ ]:
# 4-1. Agent with LCEL

from langchain.tools.render import render_text_description
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser

prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
llm_with_stop = chat_llm.bind(stop=["\nObservation"])

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [ ]:
# 4-2. 질문

query1 = "갤럭시 최신 핸드폰 기종 뭐야?"
query2 = "What is the latest Galaxy phone model?"
query3 = "구글 최신 생성형 인공지능 모델을 알려줘"
query4 = "Please tell me about Google’s latest generative artificial intelligence model."

agent_executor.invoke({"input": query1})

### CSV 데이터 분석 Agent

In [ ]:
# 파일 불러오기
import pandas as pd
df = pd.read_csv("titanic.csv")
df.head()

In [ ]:
from langchain_experimental.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(chat_llm,
                                      df=df,
                                      allow_dangerous_code=True,
                                      verbose=True)

In [ ]:
query = "전체 승객 중 1등급에 탑승한 20세 미만 여자의 생존율은 몇%야?"
agent.invoke(query)

In [ ]:
agent.invoke("어떤 데이터에 이상치가 존재하는지 확인하기 위해 그래프 그려서 확인해줘")

In [ ]:
agent.invoke("""
1등급, 2등급, 3등급 선실등급에 탑승한 승객의 생존율을 세로 막대 그래프로 그려줘.
각 등급별 남여 성별로 데이터를 나누고, 여성은 주황색/남성은 초록색으로 표시해줘
""")